In [2]:
import numpy as np 
import torch
import torch.nn as nn

from micrograd.core import Tensor
from micrograd.nn import NLLLoss

In [165]:
from numpy.lib.stride_tricks import sliding_window_view, as_strided
from itertools import product

w, h = 6, 6
x = np.arange(w*h).reshape(h, w); display(x)
u = sliding_window_view(x, (3, 3))

# u = u.reshape(-1, 3,3,3).reshape(3, 3, 3, 3)
# u = u.reshape(3, 3, -1).reshape(3, 3, 3, 3)

print(np.array(u.strides)/8, np.array(x.strides)/8)

u = u[::, ::, :, :]
# display(u)

u = np.block(list(list(x) for x in u))
u

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

[6. 1. 6. 1.] [6. 1.]


array([[ 0,  1,  2,  1,  2,  3,  2,  3,  4,  3,  4,  5],
       [ 6,  7,  8,  7,  8,  9,  8,  9, 10,  9, 10, 11],
       [12, 13, 14, 13, 14, 15, 14, 15, 16, 15, 16, 17],
       [ 6,  7,  8,  7,  8,  9,  8,  9, 10,  9, 10, 11],
       [12, 13, 14, 13, 14, 15, 14, 15, 16, 15, 16, 17],
       [18, 19, 20, 19, 20, 21, 20, 21, 22, 21, 22, 23],
       [12, 13, 14, 13, 14, 15, 14, 15, 16, 15, 16, 17],
       [18, 19, 20, 19, 20, 21, 20, 21, 22, 21, 22, 23],
       [24, 25, 26, 25, 26, 27, 26, 27, 28, 27, 28, 29],
       [18, 19, 20, 19, 20, 21, 20, 21, 22, 21, 22, 23],
       [24, 25, 26, 25, 26, 27, 26, 27, 28, 27, 28, 29],
       [30, 31, 32, 31, 32, 33, 32, 33, 34, 33, 34, 35]])

In [3]:
a = np.ones((2, 3))
b = np.ones((10, 3, 5))

y = a @ b
assert all(np.allclose(y[k], a @ b[k]) for k in range(b.shape[0]))

In [4]:
a = np.ones((2, 3, 4))
b = np.ones((3, 4, 6))
y = np.tensordot(a, b, ((1, 2), (0, 1)))
y = np.tensordot(a, b, 2)
y

array([[12., 12., 12., 12., 12., 12.],
       [12., 12., 12., 12., 12., 12.]])

In [5]:
x = np.arange(3).reshape((1, 3, 1))
y = np.broadcast_to(x, (10, 5, 3, 7))

y.sum(axis=(1, 2)).shape

def get_expanded_axes(xs, ys):
    # if y = np.broadcast_to(x, yshape), return the axes of y that are expanded
    # from x
    s = tuple(1 if len(ys)-i > len(xs) else xs[len(ys)-i-1] for i in range(len(ys)))
    print(tuple(i for i in range(len(ys)) if s[i] != ys[i]))


get_expanded_axes((1, 3, 1), (10, 5, 3, 7))

np.pad((5, 2, 3), (3, 0), constant_values=1)

(0, 1, 3)


array([1, 1, 1, 5, 2, 3])

In [6]:
import torch 

x = torch.ones((3,), dtype=torch.float32, requires_grad=True)
y = torch.ones((10, 3), dtype=torch.float32, requires_grad=True)

z = x * y
z.log().sum().backward()
x.grad

tensor([10., 10., 10.])

In [7]:
x = Tensor(np.arange(10)).broadcast((5, 10))
y = x.logsoftmax(axis=-1)
yhat = Tensor(np.array([0,1,2,3,4]))
loss = NLLLoss()(y, yhat)
loss.backward()
x.grad


array([[-1.9998440e-01,  4.2412492e-05,  1.1528910e-04,  3.1338827e-04,
         8.5187762e-04,  2.3156437e-03,  6.2945718e-03,  1.7110420e-02,
         4.6510942e-02,  1.2642986e-01],
       [ 1.5602684e-05, -1.9995759e-01,  1.1528910e-04,  3.1338827e-04,
         8.5187762e-04,  2.3156437e-03,  6.2945718e-03,  1.7110420e-02,
         4.6510942e-02,  1.2642986e-01],
       [ 1.5602684e-05,  4.2412492e-05, -1.9988471e-01,  3.1338827e-04,
         8.5187762e-04,  2.3156437e-03,  6.2945718e-03,  1.7110420e-02,
         4.6510942e-02,  1.2642986e-01],
       [ 1.5602684e-05,  4.2412492e-05,  1.1528910e-04, -1.9968662e-01,
         8.5187762e-04,  2.3156437e-03,  6.2945718e-03,  1.7110420e-02,
         4.6510942e-02,  1.2642986e-01],
       [ 1.5602684e-05,  4.2412492e-05,  1.1528910e-04,  3.1338827e-04,
        -1.9914812e-01,  2.3156437e-03,  6.2945718e-03,  1.7110420e-02,
         4.6510942e-02,  1.2642986e-01]], dtype=float32)

In [30]:
x = torch.tensor(np.broadcast_to(np.arange(10), (5,10)), dtype=torch.float32, requires_grad=True)
yhat = torch.tensor([0,1,2,3,4])

# loss = nn.NLLLoss()(nn.LogSoftmax(dim=-1)(x), yhat)
# loss.backward()

loss = nn.CrossEntropyLoss()(x, yhat)
loss.backward()
x.grad

tensor([[-1.9998e-01,  4.2412e-05,  1.1529e-04,  3.1339e-04,  8.5188e-04,
          2.3156e-03,  6.2946e-03,  1.7110e-02,  4.6511e-02,  1.2643e-01],
        [ 1.5603e-05, -1.9996e-01,  1.1529e-04,  3.1339e-04,  8.5188e-04,
          2.3156e-03,  6.2946e-03,  1.7110e-02,  4.6511e-02,  1.2643e-01],
        [ 1.5603e-05,  4.2412e-05, -1.9988e-01,  3.1339e-04,  8.5188e-04,
          2.3156e-03,  6.2946e-03,  1.7110e-02,  4.6511e-02,  1.2643e-01],
        [ 1.5603e-05,  4.2412e-05,  1.1529e-04, -1.9969e-01,  8.5188e-04,
          2.3156e-03,  6.2946e-03,  1.7110e-02,  4.6511e-02,  1.2643e-01],
        [ 1.5603e-05,  4.2412e-05,  1.1529e-04,  3.1339e-04, -1.9915e-01,
          2.3156e-03,  6.2946e-03,  1.7110e-02,  4.6511e-02,  1.2643e-01]])